# Road signs classifier

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [4]:
torch.__version__

'1.4.0'